In [ ]:
import os
import pandas
import numpy
import re

In [ ]:
type(dfb['cookies'].str.replace(r'["]','')[0]) # замена символов в квадратных скобках

In [ ]:
dfb['cookies'].str.split(',',expand=True) # разбить строковое значение в столбце и разнести по разным столбцам

In [ ]:
# пример создания столбца датафрейма из КОНКРЕТНОГО ключа словаря, находящегося в другом столбце
dfb["new"] = (dfb["headers"].apply(lambda x: x.get("Via") if isinstance(x, dict) else np.nan))